In [ ]:
!pip install timm

In [ ]:
import torch
import torchvision
import timm
from torch import nn, optim
from torch.utils.data import DataLoader
import os
from torchvision import datasets, transforms
from google.colab import drive
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
zip_path = "/content/drive/My Drive/wheat_data.zip"

In [ ]:
extract_path = "/content/wheat_data"

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

# Check it's there
print(os.listdir(extract_path))

^ 3 min to run

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

base_path = "/content/wheat_data/wheat_data"  # Fixed path

train_dir = os.path.join(base_path, "train")
valid_dir = os.path.join(base_path, "valid")
test_dir  = os.path.join(base_path, "test")


# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform)
test_dataset  = datasets.ImageFolder(test_dir,  transform=transform)

# Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
from torchvision import models
import torch.nn as nn

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
model = model.to(device)

In [ ]:
!pip install tqdm
from tqdm.notebook import tqdm

In [ ]:
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Early stopping setup
best_val_loss = float('inf')
patience = 3
counter = 0

In [ ]:
epochs = 20
best_val_loss = float('inf')
patience = 5
counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        loop = tqdm(valid_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            loop.set_postfix(val_loss=loss.item())

    val_loss /= len(valid_loader)

    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    # Early stopping logic (no saving)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        print(f"Early stopping patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            break

# Model evaluation

In [ ]:
model.eval()
# set model to evaluation mode

correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating on Test Set"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
from collections import defaultdict

model.eval()

all_preds = []
all_labels = []
class_correct = defaultdict(int)
class_total = defaultdict(int)

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating for Confusion Matrix"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        for label, pred in zip(labels, predicted):
            class_total[label.item()] += 1
            if label == pred:
                class_correct[label.item()] += 1

In [ ]:
report_dict = classification_report(
    all_labels,
    all_preds,
    target_names=train_dataset.classes,
    output_dict=True,
    zero_division=0
)

In [ ]:
# Convert report dict to DataFrame
metrics_df = pd.DataFrame(report_dict).transpose()

# Keep only class rows (filter out 'accuracy', 'macro avg', etc.)
metrics_df = metrics_df.loc[train_dataset.classes]

# Round for readability
metrics_df = metrics_df.round(2)

# Print the DataFrame
print(metrics_df)

In [ ]:
# Store results in a list of dictionaries
accuracy_data = []

for idx, class_name in enumerate(train_dataset.classes):
    accuracy = 100 * class_correct[idx] / class_total[idx] if class_total[idx] > 0 else 0.0
    accuracy_data.append({
        "Class": class_name,
        "Accuracy (%)": round(accuracy, 2),
        "Correct": class_correct[idx],
        "Total": class_total[idx]
    })

# Convert to DataFrame
accuracy_df = pd.DataFrame(accuracy_data)

# Print the DataFrame
print(accuracy_df)
